In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras import callbacks
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/Datasets/alzheimers_prediction_dataset.csv')

Mounted at /content/drive


In [3]:
data.head()

,Country,Age,Gender,Education Level,BMI,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,...,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living,Alzheimer’s Diagnosis
0,Spain,90,Male,1,33.0,Medium,Never,Occasionally,No,No,...,Healthy,High,Retired,Single,No,Low,Medium,High,Urban,No
1,Argentina,72,Male,7,29.9,Medium,Former,Never,No,No,...,Healthy,Medium,Unemployed,Widowed,No,High,Low,High,Urban,No
2,South Africa,86,Female,19,22.9,High,Current,Occasionally,No,Yes,...,Average,Medium,Employed,Single,No,Low,Medium,High,Rural,No
3,China,53,Male,17,31.2,Low,Never,Regularly,Yes,No,...,Healthy,Medium,Retired,Single,No,High,Medium,Low,Rural,No
4,Sweden,58,Female,3,30.0,High,Former,Never,Yes,No,...,Unhealthy,High,Employed,Married,No,Low,Medium,High,Rural,No


In [4]:
data.tail()

,Country,Age,Gender,Education Level,BMI,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,...,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living,Alzheimer’s Diagnosis
74278,Russia,60,Female,3,22.6,High,Former,Never,No,No,...,Average,High,Unemployed,Widowed,No,Medium,High,Medium,Rural,No
74279,UK,58,Male,18,30.6,Low,Never,Occasionally,Yes,No,...,Average,Medium,Unemployed,Single,No,Medium,High,High,Rural,No
74280,Spain,57,Female,13,28.2,Medium,Never,Regularly,No,No,...,Healthy,Low,Employed,Single,Yes,High,Low,Low,Rural,No
74281,Brazil,73,Female,7,29.0,Low,Never,Regularly,No,No,...,Healthy,Low,Employed,Widowed,No,Low,Low,High,Rural,No
74282,Norway,57,Female,1,31.7,Low,Current,Regularly,No,No,...,Average,Low,Unemployed,Single,No,Low,Medium,Medium,Urban,No


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74283 entries, 0 to 74282
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Country                               74283 non-null  object 
 1   Age                                   74283 non-null  int64  
 2   Gender                                74283 non-null  object 
 3   Education Level                       74283 non-null  int64  
 4   BMI                                   74283 non-null  float64
 5   Physical Activity Level               74283 non-null  object 
 6   Smoking Status                        74283 non-null  object 
 7   Alcohol Consumption                   74283 non-null  object 
 8   Diabetes                              74283 non-null  object 
 9   Hypertension                          74283 non-null  object 
 10  Cholesterol Level                     74283 non-null  object 
 11  Family History 

In [6]:
data.describe()

,Age,Education Level,BMI,Cognitive Test Score
count,74283.000000,74283.000000,74283.000000,74283.000000
mean,71.964703,9.487514,26.780639,64.654241
std,12.980748,5.757020,4.764679,20.153247
min,50.000000,0.000000,18.500000,30.000000
25%,61.000000,4.000000,22.700000,47.000000
50%,72.000000,9.000000,26.800000,65.000000
75%,83.000000,14.000000,30.900000,82.000000
max,94.000000,19.000000,35.000000,99.000000


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
label_encoders = {}
categorical_cols = ['Country', 'Gender', 'Education Level', 'Physical Activity Level', 'Smoking Status',
                    'Alcohol Consumption', 'Diabetes', 'Hypertension','Cholesterol Level','Family History of Alzheimer’s','Depression Level','Sleep Quality','Dietary Habits', 'Air Pollution Exposure',
                    'Employment Status', 'Marital Status', 'Genetic Risk Factor (APOE-ε4 allele)',
                    'Social Engagement Level', 'Income Level', 'Stress Levels', 'Urban vs Rural Living']

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Save encoders for later use

In [9]:
data.head()

,Country,Age,Gender,Education Level,BMI,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,...,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living,Alzheimer’s Diagnosis
0,16,90,1,1,33.0,2,2,1,0,0,...,1,0,1,1,0,1,2,0,1,No
1,0,72,1,7,29.9,2,1,0,0,0,...,1,2,2,2,0,0,1,0,1,No
2,14,86,0,19,22.9,0,0,1,0,1,...,0,2,0,1,0,1,2,0,0,No
3,4,53,1,17,31.2,1,2,2,1,0,...,1,2,1,1,0,0,2,1,0,No
4,17,58,0,3,30.0,0,1,0,1,0,...,2,0,0,0,0,1,2,0,0,No


In [10]:
# Encode the target variable (Binary classification: Yes → 1, No → 0)
data['Alzheimer’s Diagnosis'] = data['Alzheimer’s Diagnosis'].map({'Yes': 1, 'No': 0})

In [11]:
data.head()

,Country,Age,Gender,Education Level,BMI,Physical Activity Level,Smoking Status,Alcohol Consumption,Diabetes,Hypertension,...,Dietary Habits,Air Pollution Exposure,Employment Status,Marital Status,Genetic Risk Factor (APOE-ε4 allele),Social Engagement Level,Income Level,Stress Levels,Urban vs Rural Living,Alzheimer’s Diagnosis
0,16,90,1,1,33.0,2,2,1,0,0,...,1,0,1,1,0,1,2,0,1,0
1,0,72,1,7,29.9,2,1,0,0,0,...,1,2,2,2,0,0,1,0,1,0
2,14,86,0,19,22.9,0,0,1,0,1,...,0,2,0,1,0,1,2,0,0,0
3,4,53,1,17,31.2,1,2,2,1,0,...,1,2,1,1,0,0,2,1,0,0
4,17,58,0,3,30.0,0,1,0,1,0,...,2,0,0,0,0,1,2,0,0,0


In [12]:
# Split data into features and target
X = data.drop(columns=['Alzheimer’s Diagnosis'])
y = data['Alzheimer’s Diagnosis']

In [13]:
# Normalize numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
# Split into train & test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Build ANN Model using Keras
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))  # Prevent overfitting
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification output

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.6649 - loss: 0.6036 - val_accuracy: 0.7147 - val_loss: 0.5463
Epoch 2/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7098 - loss: 0.5560 - val_accuracy: 0.7160 - val_loss: 0.5455
Epoch 3/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7142 - loss: 0.5518 - val_accuracy: 0.7173 - val_loss: 0.5415
Epoch 4/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.7185 - loss: 0.5481 - val_accuracy: 0.7211 - val_loss: 0.5421
Epoch 5/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7189 - loss: 0.5478 - val_accuracy: 0.7216 - val_loss: 0.5422
Epoch 6/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7189 - loss: 0.5464 - val_accuracy: 0.7226 - val_loss: 0.5410
Epoch 7/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7214 - loss: 0.5439 - val_accuracy: 0.7218 - val_loss: 0.5393
Epoch 8/50
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7224 - loss: 0.5

In [18]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc:.2f}")

465/465 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7249 - loss: 0.5389

Test Accuracy: 0.72


In [19]:
# Predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

465/465 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [20]:
# Print classification report (optional)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.73      0.76      8719
           1       0.65      0.71      0.68      6138

    accuracy                           0.72     14857
   macro avg       0.72      0.72      0.72     14857
weighted avg       0.73      0.72      0.72     14857

